# 410_prep_RQ2_Dataframe
## Purpose 
In this notebook we prepare a dataframe for our Research Question 2.  
## Datasets 
* _Input_: Joined1617.csv,Joined1516.csv,Joined1415.csv,Joined1314.csv,Joined1213.csv
* _Output_: RQ2.csv

In [1]:
import math
import os.path
import numpy as np
import pandas as pd

#### Reading in our cleaned Joined datasets from 16-17 to 12-13.

In [2]:
J16_17 = pd.read_csv("../../data/prep/Joined1617.csv")
J15_16 = pd.read_csv("../../data/prep/Joined1516.csv")
J14_15 = pd.read_csv("../../data/prep/Joined1415.csv")
J13_14 = pd.read_csv("../../data/prep/Joined1314.csv")
J12_13 = pd.read_csv("../../data/prep/Joined1213.csv")

## Choosing which columns are needed for the Research Question
* index refers to all columns that have a fixed figure.
* fantasyinfo contains columns for the Fantasy Football datasets that change every year.

In [3]:
index = ['Players','Position','league','age','nationality','photo','overall','skill_moves','pac','def','pas','dri','eur_value','eur_wage']
fantasyinfo = ['Apps','Points','Start','Off','Goals','A','CS']

### Concatting each years fantasyinfo using the year as a key

In [21]:
s1 = J16_17.drop_duplicates(index).set_index(index)[fantasyinfo]
s2 = J15_16.drop_duplicates(index).set_index(index)[fantasyinfo]
s3 = J14_15.drop_duplicates(index).set_index(index)[fantasyinfo]
s4 = J13_14.drop_duplicates(index).set_index(index)[fantasyinfo]
s5 = J12_13.drop_duplicates(index).set_index(index)[fantasyinfo]

RQ2 = pd.concat([s1,s2,s3,s4,s5], axis=1, keys=('16/17','15/16','14/15','13/14','12/13')).fillna(0).astype(float).reset_index()


## Creating New Columns
* **Homegrown** - contains players who are playing in a league in their home country. ie: An English Player Playing in England.<br><br>
* **Abroad** - contains all players that live in the location of the Top 5 Leagues (England,France,Italy,Spain,Germany) that play in one of the other Top 5 Leagues.  Examples: <br> A French player playing in Germany is said to be an Abroad player. <br> A Home player is the same as a homegrown player, ie: A French player playing in France.<br><br>
* **Overall Apps** - contains the total number of appearances a player has made.<br><br>
* **Starts/Apps** - is the probability of a player starting a game. It is calculated by the total number of starts divided by the overall appearances of a player.<br><br>
* **Subbed Off/Apps** - is the probability of a player being subbed off in a game. It is calculated by the total number of times subbed off divided by the overall appearances of a player.<br><br>
* **Average Form** - is the total average form of a player. This is calculated by dividing a players total points by their overall appearances producing an average points per game.<br><br>
* **Goals/Apps** - is the probablity of a player scoring a goal in a match. This is calculated by dividing a players total goals by their overall appearances.<br><br>
* **Assists/Apps** - is the probablity of a player assisting a goal in a match. This is calculated by dividing a players total assists by their overall appearances.<br><br>
* **Clean Sheets/Apps** - is the probablity of a player keeping a clean sheet in a match. This is calculated by dividing a players total clean sheets by their overall appearances. This is only applicable to Goalkeepers and Defenders.

In [22]:
RQ2['Homegrown'] = (RQ2['nationality'] =='England') & (RQ2['league'] == 'English Premier League') |(RQ2['nationality'] =='Germany') & (RQ2['league'] == 'German Bundesliga')|(RQ2['nationality'] =='Spain') & (RQ2['league'] == 'Spanish Primera Division')|(RQ2['nationality'] =='France') & (RQ2['league'] == 'French Ligue 1')|(RQ2['nationality'] =='Italy') & (RQ2['league'] == 'Italian Serie A')

In [23]:
RQ2['Abroad'] = (RQ2['nationality'] =='England') & (RQ2['league'].isin(['German Bundesliga','Spanish Primera Division','French Ligue 1','Italian Serie A'])) | (RQ2['nationality'] =='Germany') & (RQ2['league'].isin(['English Premier League','Spanish Primera Division','French Ligue 1','Italian Serie A']))|(RQ2['nationality'] =='Spain') & (RQ2['league'].isin(['English Premier League','German Bundesliga','French Ligue 1','Italian Serie A']))|(RQ2['nationality'] =='France') & (RQ2['league'].isin(['English Premier League','German Bundesliga','Spanish Primera Division','Italian Serie A']))|(RQ2['nationality'] =='Italy') & (RQ2['league'].isin(['English Premier League','German Bundesliga','Spanish Primera Division','French Ligue 1']))

In [24]:
RQ2['Overall Apps'] = RQ2['16/17']['Apps']+RQ2['15/16']['Apps']+RQ2['14/15']['Apps']+RQ2['13/14']['Apps']+RQ2['12/13']['Apps']

In [25]:
RQ2['Starts/Apps'] = (RQ2['16/17']['Start']+RQ2['15/16']['Start']+RQ2['14/15']['Start']+RQ2['13/14']['Start']+RQ2['12/13']['Start'])/RQ2['Overall Apps']

In [26]:
RQ2['Subbed Off/Apps'] = (RQ2['16/17']['Off']+RQ2['15/16']['Off']+RQ2['14/15']['Off']+RQ2['13/14']['Off']+RQ2['12/13']['Off'])/RQ2['Overall Apps']

In [27]:
RQ2['Average Form'] = (RQ2['16/17']['Points']+RQ2['15/16']['Points']+RQ2['14/15']['Points']+RQ2['13/14']['Points']+RQ2['12/13']['Points'])/RQ2['Overall Apps']

In [28]:
RQ2['Goals/Apps'] = (RQ2['16/17']['Goals']+RQ2['15/16']['Goals']+RQ2['14/15']['Goals']+RQ2['13/14']['Goals']+RQ2['12/13']['Goals'])/RQ2['Overall Apps']

In [29]:
RQ2['Assists/Apps'] = (RQ2['16/17']['A']+RQ2['15/16']['A']+RQ2['14/15']['A']+RQ2['13/14']['A']+RQ2['12/13']['A'])/RQ2['Overall Apps']

In [30]:
RQ2['Clean Sheets/Apps'] = (RQ2['16/17']['CS']+RQ2['15/16']['CS']+RQ2['14/15']['CS']+RQ2['13/14']['CS']+RQ2['12/13']['CS'])/RQ2['Overall Apps']

## Filtering
**For reproducibility purposes**
* Removes players with an Average Form over 50 as this is impossible
* Removes players with less than 0 appearances.

In [31]:
RQ2= RQ2[RQ2['Average Form']<50]
RQ2 = RQ2[RQ2['Overall Apps']>=0]

## Tidying Up
* First checking what columns are contain in the dataframe.
* Finally choosing all the relevant columns to the questions we wish to ask.

In [32]:
list(RQ2)

[('Players', ''),
 ('Position', ''),
 ('league', ''),
 ('age', ''),
 ('nationality', ''),
 ('photo', ''),
 ('overall', ''),
 ('skill_moves', ''),
 ('pac', ''),
 ('def', ''),
 ('pas', ''),
 ('dri', ''),
 ('eur_value', ''),
 ('eur_wage', ''),
 ('16/17', 'Apps'),
 ('16/17', 'Points'),
 ('16/17', 'Start'),
 ('16/17', 'Off'),
 ('16/17', 'Goals'),
 ('16/17', 'A'),
 ('16/17', 'CS'),
 ('15/16', 'Apps'),
 ('15/16', 'Points'),
 ('15/16', 'Start'),
 ('15/16', 'Off'),
 ('15/16', 'Goals'),
 ('15/16', 'A'),
 ('15/16', 'CS'),
 ('14/15', 'Apps'),
 ('14/15', 'Points'),
 ('14/15', 'Start'),
 ('14/15', 'Off'),
 ('14/15', 'Goals'),
 ('14/15', 'A'),
 ('14/15', 'CS'),
 ('13/14', 'Apps'),
 ('13/14', 'Points'),
 ('13/14', 'Start'),
 ('13/14', 'Off'),
 ('13/14', 'Goals'),
 ('13/14', 'A'),
 ('13/14', 'CS'),
 ('12/13', 'Apps'),
 ('12/13', 'Points'),
 ('12/13', 'Start'),
 ('12/13', 'Off'),
 ('12/13', 'Goals'),
 ('12/13', 'A'),
 ('12/13', 'CS'),
 ('Homegrown', ''),
 ('Abroad', ''),
 ('Overall Apps', ''),
 ('Starts

In [33]:
RQ2 = RQ2[['Players','Position','league','age','nationality','overall','photo','skill_moves','pac','def','pas','dri','eur_value','eur_wage','Homegrown','Abroad','Starts/Apps','Subbed Off/Apps','Overall Apps','Average Form','Goals/Apps','Assists/Apps','Clean Sheets/Apps']]
RQ2.head(5)

,Players,Position,league,age,nationality,overall,photo,skill_moves,pac,def,...,eur_wage,Homegrown,Abroad,Starts/Apps,Subbed Off/Apps,Overall Apps,Average Form,Goals/Apps,Assists/Apps,Clean Sheets/Apps
,,,,,,,,,,,...,,,,,,,,,,
0,Aaron Cresswell,Defender,English Premier League,27,England,76,https://cdn.sofifa.org/18/players/189615.png,2,76,73,...,83000.0,True,False,0.980198,0.039604,101.0,2.821782,0.039604,0.099010,0.247525
1,Aaron Hughes,Defender,Scottish Premiership,37,Northern Ireland,71,https://cdn.sofifa.org/18/players/17725.png,2,23,72,...,5000.0,False,False,0.918919,0.000000,37.0,1.216216,0.000000,0.000000,0.162162
2,Aaron Hunt,Midfielder,German Bundesliga,30,Germany,76,https://cdn.sofifa.org/18/players/158138.png,3,68,39,...,38000.0,True,False,0.788136,0.279661,118.0,3.050847,0.211864,0.118644,0.000000
3,Aaron Lennon,Midfielder,English Premier League,30,England,77,https://cdn.sofifa.org/18/players/152747.png,3,83,38,...,97000.0,True,False,0.808333,0.391667,120.0,2.633333,0.100000,0.125000,0.000000
4,Aaron Leya Iseka,Forward,Belgian First Division A,19,Belgium,68,https://cdn.sofifa.org/18/players/223929.png,3,79,21,...,9000.0,False,False,0.375000,0.375000,8.0,2.125000,0.000000,0.250000,0.000000


#### Saving to csv file in data/analysis

In [34]:
RQ2.to_csv('../../data/analysis/RQ2.csv')